In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
song=pd.read_csv('/kaggle/input/spotifyclassification/data.csv')
song.head()

In [ ]:
song.drop('Unnamed: 0',axis=1,inplace=True)
song.head()

In [ ]:
song.info()

In [ ]:
song.describe()

In [ ]:
song.target.value_counts()

# Duplicate Check 

In [ ]:
songd=song.copy()
songd.drop_duplicates(subset=None,inplace=True)
songd.shape

In [ ]:
song.shape

In [ ]:
song=songd
del songd
song.shape

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(song.corr(), annot=True,cmap='RdBu')
plt.show()

In [ ]:
song.columns

In [ ]:
left = song[['target','song_title', 'artist']]
left.head()

In [ ]:
song.drop(['target','song_title', 'artist'],axis=1,inplace=True)
song.head()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def vif_show(X_vif):
    vif = pd.DataFrame()
    vif['Features'] = X_vif.columns
    vif['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    print(vif)
    print()
    if(vif.iloc[0,1] > 5.0 ):
        col = vif.iloc[0,0]
        X_vif.drop([vif.iloc[0,0]],axis =1, inplace = True)
        print("After removing \""+ col + "\" from datafame")
        vif_show(X_vif)

vif_show(song)

In [ ]:
song.columns

In [ ]:
song = pd.concat([song,left],axis = 1)
song.head()

In [ ]:
song.shape

In [ ]:
#Note: - We reduced 9 Columns from original 15 columns, which will reduce training time  for teh model

In [ ]:
X=song.drop('target',axis=1)
y=song['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.80,stratify=y, random_state = 1)

In [ ]:
X_train.info()

In [ ]:
song.head()

In [ ]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()

In [ ]:
obj=list(np.where(X_train.dtypes == np.object)[0])
obj

In [ ]:
model.fit(X_train,y_train,cat_features=obj)

In [ ]:
import sklearn.metrics
print(sklearn.metrics.classification_report(y_test,model.predict(X_test)))

In [ ]:
# predict result
X_test['target']= model.predict(X_test)
X_test.head()

In [ ]:
# Factors which decide that I will like the song or not 

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_train.columns,
    "Imp": model.feature_importances_})
imp_df.sort_values(by="Imp", ascending=False)